# Linear Regression

Housing Prices Competition for Kaggle Learn Users:
* https://www.kaggle.com/c/home-data-for-ml-course/data

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
house_data = pd.read_csv('https://raw.githubusercontent.com/benjum/UCLAX-23W-ML/main/Weeks/Week02/data/home-train.csv')

In [ ]:
house_data.head()

In [ ]:
house_data.info()

In [ ]:
h2 = house_data[['OverallQual','OverallCond','BedroomAbvGr','GrLivArea','SalePrice']].copy()

In [ ]:
h2.sample(10)

In [ ]:
sns.regplot(data=h2, x='OverallQual', y='SalePrice')

In [ ]:
sns.regplot(data=h2, x='OverallCond', y='SalePrice')

In [ ]:
sns.regplot(data=h2, x='BedroomAbvGr', y='SalePrice')

In [ ]:
sns.regplot(data=h2, x='GrLivArea', y='SalePrice')

### Using one feature for simple linear regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import StandardScaler

In [ ]:
X = h2[['GrLivArea']]
y = h2['SalePrice']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#scaler = StandardScaler()

In [ ]:
#X_train = scaler.fit_transform(X_train)

In [ ]:
#X_test = scaler.transform(X_test)

In [ ]:
linear_regression = LinearRegression()
model = linear_regression.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
df = pd.DataFrame({'test': y_test, 'predicted': y_pred})

In [ ]:
df.sample(10)

### Regression line

In [ ]:
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, c='r')

In [ ]:
print("Training score : ", linear_regression.score(X_train, y_train))
print("Testing score : ", linear_regression.score(X_test, y_test))

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
r2score = r2_score(y_test, y_pred)
msescore = mean_squared_error(y_test, y_pred)

In [ ]:
print("Testing score R2 : ", r2score)
print("Testing score StdDev : ", np.sqrt(msescore))

In [ ]:
theta_0 = linear_regression.coef_
theta_0

In [ ]:
intercept = linear_regression.intercept_
intercept

In [ ]:
plt.plot(y_pred, label="Prediction")
plt.plot(y_test.values, label="Actual")
plt.legend()

In [ ]:
plt.plot(y_test.values, y_pred, 'ko')
plt.plot([0,600000],[0,600000])

## Using statsmodels and OLS instead of scikit-learn

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train[:5]

In [ ]:
X_train = sm.add_constant(X_train)

In [ ]:
X_train[:5]

In [ ]:
model = sm.OLS(y_train, X_train).fit()

In [ ]:
print(model.summary())

In [ ]:
theta_0, intercept

## Multiple Regression 

In [ ]:
house_data.shape

In [ ]:
target = house_data['SalePrice']
features = house_data.drop('SalePrice', axis=1)

In [ ]:
features.info()

I'm only going to keep numerical columns for now.

In [ ]:
features = features.select_dtypes(include='number').copy()

And I'm going to drop the columns that don't have 1460 non-null values.

In [ ]:
features = features.drop(['GarageYrBlt','MasVnrArea','LotFrontage'],axis=1)

In [ ]:
features.columns

## Yellowbrick

* "Yellowbrick extends the Scikit-Learn API to make model selection and hyperparameter tuning easier. Under the hood, it’s using Matplotlib."
* https://www.scikit-yb.org/en/latest/

In [ ]:
# Need to install yellowbrick if you don't have it

In [ ]:
from yellowbrick.target import FeatureCorrelation

In [ ]:
visualizer = FeatureCorrelation(labels = list(features.columns), sort=True)
visualizer.fit(features, target)
visualizer.show()

### Select K-Best features to predict price of houses

Scikit-Learn has methods for selecting the "best" features.  For example, the following will use `f_regression` to do "univariate linear regression tests returning F-statistic and p-values" in order to select a set of best features.
* [feature_selection with f_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
best6 = SelectKBest(f_regression, k=6).fit(features, target)

In [ ]:
best6

In [ ]:
help(best6)

In [ ]:
best6.get_feature_names_out()

In [ ]:
best6.get_support()

In [ ]:
bestcolumns = features.columns[best6.get_support()]
bestcolumns

In [ ]:
myfeatures = features[bestcolumns]
myfeatures.head()

In [ ]:
myfeatures.describe()

### Aside

When we consider feature engineering later, we'll want to consider whether some of the variables are dependent.  And exactly what happens when we have multicollinearity of the variables.

Multicollinearity may not have an effect on the predictive power of our model, but it can affect the variance of our coefficient estimates, lead to larger confidence intervals, and make it more difficult to interpret the predictors of our final model.

Here we briefly show how to identify whether variables might suffer from collinearity by measuring the variance inflation factor (VIF) -- values of 1 are ignorable, < 5 are reasonable, and > 5 should be dealt with.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
myfeatures.values

In [ ]:
vif = pd.DataFrame()
vif['VIF'] = [round(variance_inflation_factor(myfeatures.values, i), 1) for i in range(myfeatures.shape[1])]
vif['variable'] = myfeatures.columns

In [ ]:
vif.sort_values(by='VIF', ascending=False)

### Scaling

Scaling of feature values is another feature engineering concept that we will need to contend with.

Feature scaling can be necessary for applying certain algorithms, it may not matter at all in other cases, sometimes it can make algorithms run faster, it can give a better error surface shape, it can prevent optimization algorithms from getting stuck in local minima, it can reduce the collinearity of two variables, ....

Here we have features that have very different scales, and we'll show scaling as a method to make their scales uniform, but we will return to this in more detail throughout the course.

In [ ]:
from sklearn.preprocessing import scale

In [ ]:
X = pd.DataFrame(data=scale(myfeatures), columns=myfeatures.columns)
y = target

In [ ]:
X.describe()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Wait!  Danger:**  we've scaled our data before doing the test/train split.  Information about the training set may therefore have leaked into the test set.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = pd.DataFrame(data=myfeatures, columns=myfeatures.columns)
y = target

In [ ]:
X.describe()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train.mean())
print(X_train.std())
print(X_test.mean())
print(X_test.std())

In [ ]:
print(np.concatenate((X_train,X_test)).mean())
print(np.concatenate((X_train,X_test)).std())

Ok, better.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_regression = LinearRegression()

In [ ]:
linear_regression.fit(X_train, y_train)

In [ ]:
y_pred = linear_regression.predict(X_test)

In [ ]:
df = pd.DataFrame({'test': y_test, 'Predicted': y_pred})

In [ ]:
df.head()

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
score = linear_regression.score(X_train, y_train)
r2score = r2_score(y_test, y_pred)
stddevscore = np.sqrt(mean_squared_error(y_test, y_pred))

print('Score: {}'.format(score))
print('r2_score: {}'.format(r2score))
print('stddev_score: {}'.format(stddevscore))

In [ ]:
linear_regression.coef_

In [ ]:
linear_regression.intercept_

## The statsmodels way

In [ ]:
X_train = sm.add_constant(X_train)

In [ ]:
ols = sm.OLS(y_train, X_train)
model = ols.fit()
y_pred = model.predict(X_train)

model.summary()

In [ ]:
linear_regression.intercept_

In [ ]:
linear_regression.coef_

## Exercises

* What happens if you don't do any scaling above for the case with best6 features.  Does the performance change?

* Notice that two values don't appear to be statistically significant.... can you get just as good a result for only picking 4 values?

* Also, try checking out a slightly different method for selecting features: mutual_info

* Try using the full training set for training and using the 'test.csv' data to do the test.

In [ ]:
visualizer = FeatureCorrelation(method='mutual_info-regression',
                                labels = list(features.columns),
                                sort=True)
visualizer.fit(features, target)
visualizer.show()